In [1]:
import numpy as np
import pandas as pd
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
metadata=pd.read_csv('spih_dataset_csv.csv')

metadata.describe()
metadata.head()

,pid,product_name,number_of_reviews,average_review_rating,retail_price,discounted_price,discount_percent,brand,description,product_category_tree,image,lifetime
0,SRTEH2FF9KEDEFGF,Alisha Solid Women's Cycling Shorts,15,4.9,999.0,379.0,NaN,Alisha,Key Features of Alisha Solid Women's Cycling S...,"[Clothing >> Women's Clothing >> Lingerie, Sle...",[http://img5a.flixcart.com/image/short/u/4/a/a...,20.0
1,SBEEH3QGU7MFYJFY,FabHomeDecor Fabric Double Sofa Bed,2,4.5,32157.0,22646.0,NaN,FabHomeDecor,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,[Furniture >> Living Room Furniture >> Sofa Be...,[http://img6a.flixcart.com/image/sofa-bed/j/f/...,NaN
2,SHOEH4GRSUBJGZXE,AW Bellies,17,3.9,999.0,499.0,NaN,AW,Key Features of AW Bellies Sandals Wedges Heel...,[Footwear >> Women's Footwear >> Ballerinas >>...,[http://img5a.flixcart.com/image/shoe/7/z/z/re...,NaN
3,SRTEH2F6HUZMQ6SJ,Alisha Solid Women's Cycling Shorts,1,5.0,699.0,267.0,NaN,Alisha,Key Features of Alisha Solid Women's Cycling S...,"[Clothing >> Women's Clothing >> Lingerie, Sle...",[http://img5a.flixcart.com/image/short/6/2/h/a...,NaN
4,PSOEH3ZYDMSYARJ5,Sicons All Purpose Arnica Dog Shampoo,3,4.7,220.0,210.0,NaN,Sicons,Specifications of Sicons All Purpose Arnica Do...,[Pet Supplies >> Grooming >> Skin & Coat Care ...,[http://img5a.flixcart.com/image/pet-shampoo/r...,NaN


In [3]:
metadata['description'].head(15)

0     Key Features of Alisha Solid Women's Cycling S...
1     FabHomeDecor Fabric Double Sofa Bed (Finish Co...
2     Key Features of AW Bellies Sandals Wedges Heel...
3     Key Features of Alisha Solid Women's Cycling S...
4     Specifications of Sicons All Purpose Arnica Do...
5     Key Features of Eternal Gandhi Super Series Cr...
6     Key Features of Alisha Solid Women's Cycling S...
7     FabHomeDecor Fabric Double Sofa Bed (Finish Co...
8     Key Features of dilli bazaaar Bellies, Corpora...
9     Key Features of Alisha Solid Women's Cycling S...
10    Key Features of Ladela Bellies Brand: LADELA C...
11    Key Features of Carrel Printed Women's Fabric:...
12    Specifications of Sicons All Purpose Tea Tree ...
13    Key Features of Alisha Solid Women's Cycling S...
14    Specifications of Freelance Vacuum Bottles 350...
Name: description, dtype: object

In [4]:
#temp_category_tree=metadata['product_category_tree'].split('>>')
metadata['soup']=metadata['description']+metadata['product_name']+metadata['brand']+metadata['product_category_tree']
metadata['soup']=metadata['soup'].fillna('')
metadata['soup']

0        Key Features of Alisha Solid Women's Cycling S...
1        FabHomeDecor Fabric Double Sofa Bed (Finish Co...
2        Key Features of AW Bellies Sandals Wedges Heel...
3        Key Features of Alisha Solid Women's Cycling S...
4        Specifications of Sicons All Purpose Arnica Do...
5        Key Features of Eternal Gandhi Super Series Cr...
6        Key Features of Alisha Solid Women's Cycling S...
7        FabHomeDecor Fabric Double Sofa Bed (Finish Co...
8        Key Features of dilli bazaaar Bellies, Corpora...
9        Key Features of Alisha Solid Women's Cycling S...
10       Key Features of Ladela Bellies Brand: LADELA C...
11       Key Features of Carrel Printed Women's Fabric:...
12       Specifications of Sicons All Purpose Tea Tree ...
13       Key Features of Alisha Solid Women's Cycling S...
14       Specifications of Freelance Vacuum Bottles 350...
15       Key Features of Alisha Solid Women's Cycling S...
16       FabHomeDecor Fabric Double Sofa Bed (Finish Co.

In [5]:
#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
#metadata['description'] = metadata['description'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(metadata['soup'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(10002, 18437)

In [6]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [7]:
indices = pd.Series(metadata.index, index=metadata['pid']).drop_duplicates()

In [9]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:21]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 20 most similar movies
    print(metadata['product_name'].iloc[movie_indices])
    return metadata['pid'].iloc[movie_indices]

In [10]:
print(get_recommendations('SRTEH2FF9KEDEFGF'))

3                    Alisha Solid Women's Cycling Shorts
9                    Alisha Solid Women's Cycling Shorts
6                    Alisha Solid Women's Cycling Shorts
13                   Alisha Solid Women's Cycling Shorts
15                   Alisha Solid Women's Cycling Shorts
21                   Alisha Solid Women's Cycling Shorts
978    Mynte Solid Women's Cycling Shorts, Gym Shorts...
936    Mynte Solid Women's Cycling Shorts, Gym Shorts...
939    Mynte Solid Women's Cycling Shorts, Gym Shorts...
979    Mynte Solid Women's Cycling Shorts, Gym Shorts...
965    Mynte Solid Women's Cycling Shorts, Gym Shorts...
963    Mynte Solid Women's Cycling Shorts, Gym Shorts...
945    Mynte Solid Women's Cycling Shorts, Gym Shorts...
975    Mynte Solid Women's Cycling Shorts, Gym Shorts...
941    Mynte Solid Women's Cycling Shorts, Gym Shorts...
980    Mynte Solid Women's Cycling Shorts, Gym Shorts...
966    Mynte Solid Women's Cycling Shorts, Gym Shorts...
935    Mynte Solid Women's Cycl